In [ ]:
SEED = 42
import os, random, numpy as np
os.environ["PYTHONHASHSEED"] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
# PyTorch seeding (TensorFlow is skipped to avoid environment issues)
try:
    import torch
    torch.manual_seed(SEED)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(SEED)
        torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
except Exception:
    pass


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/eds-232-ocean-chemistry-prediction-for-calcofi/sample_submission.csv
/kaggle/input/eds-232-ocean-chemistry-prediction-for-calcofi/train.csv
/kaggle/input/eds-232-ocean-chemistry-prediction-for-calcofi/test.csv


In [ ]:
train=pd.read_csv("/kaggle/input/eds-232-ocean-chemistry-prediction-for-calcofi/train.csv")
test=pd.read_csv("/kaggle/input/eds-232-ocean-chemistry-prediction-for-calcofi/test.csv")
sub=pd.read_csv("/kaggle/input/eds-232-ocean-chemistry-prediction-for-calcofi/sample_submission.csv")

In [ ]:
train

,id,Lat_Dec,Lon_Dec,NO2uM,NO3uM,NH3uM,R_TEMP,R_Depth,R_Sal,R_DYNHT,R_Nuts,R_Oxy_micromol.Kg,Unnamed: 12,PO4uM,SiO3uM,TA1.x,Salinity1,Temperature_degC,DIC
0,1,34.385030,-120.665530,0.030,33.80,0.00,7.79,323,141.2,0.642,0.00,37.40948,NaN,2.77,53.86,2287.45,34.1980,7.82,2270.17
1,2,31.418333,-121.998333,0.000,34.70,0.00,7.12,323,140.8,0.767,0.00,64.81441,NaN,2.57,52.50,2279.10,34.0740,7.15,2254.10
2,3,34.385030,-120.665530,0.180,14.20,0.00,11.68,50,246.8,0.144,0.00,180.29150,NaN,1.29,13.01,2230.80,33.5370,11.68,2111.04
3,4,33.482580,-122.533070,0.013,29.67,0.01,8.33,232,158.5,0.562,0.01,89.62595,NaN,2.27,38.98,2265.85,34.0480,8.36,2223.41
4,5,31.414320,-121.997670,0.000,33.10,0.05,7.53,323,143.4,0.740,0.05,60.03062,NaN,2.53,49.28,2278.49,34.1170,7.57,2252.62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1449,1450,33.420000,-117.901667,0.000,38.60,0.00,6.52,520,118.2,0.889,0.00,12.17673,NaN,3.11,73.10,2313.52,34.3024,6.57,2311.19
1450,1451,34.271667,-120.023333,0.330,5.90,0.00,13.21,49,290.1,0.149,0.00,224.81120,NaN,0.75,6.50,2223.31,33.3304,13.22,2048.93
1451,1452,31.410000,-121.991667,0.000,33.60,0.00,7.61,322,143.1,0.731,0.00,60.46569,NaN,2.56,49.50,2280.43,34.1269,7.64,2251.34
1452,1453,34.274270,-120.030570,0.517,1.58,0.35,14.47,30,319.0,0.110,0.35,249.28420,NaN,0.54,4.12,2223.88,33.2660,14.47,2030.03


In [ ]:
test

,id,Lat_Dec,Lon_Dec,NO2uM,NO3uM,NH3uM,R_TEMP,R_Depth,R_Sal,R_DYNHT,R_Nuts,R_Oxy_micromol.Kg,PO4uM,SiO3uM,TA1,Salinity1,Temperature_degC
0,1455,34.321666,-120.811666,0.020,24.00,0.41,9.51,101,189.9,0.258,0.41,138.838300,1.85,25.50,2244.94,33.830,9.52
1,1456,34.275000,-120.033333,0.000,25.10,0.00,9.84,102,185.2,0.264,0.00,102.709200,2.06,28.30,2253.27,33.963,9.85
2,1457,34.275000,-120.033333,0.000,31.90,0.00,6.60,514,124.1,0.874,0.00,2.174548,3.40,88.10,2316.95,34.241,6.65
3,1458,33.828333,-118.625000,0.000,0.00,0.20,19.21,1,408.1,0.004,0.20,258.674300,0.27,2.50,2240.49,33.465,19.21
4,1459,33.828333,-118.625000,0.020,19.70,0.00,10.65,100,215.5,0.274,0.00,145.839900,1.64,19.40,2238.30,33.720,10.66
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
480,1935,31.418030,-121.989970,0.000,0.00,0.01,21.20,2,465.3,0.009,0.01,229.490000,0.28,1.89,2235.34,33.380,21.20
481,1936,31.418030,-121.989970,0.000,24.50,0.00,8.54,232,171.3,0.689,0.00,153.330000,1.81,29.40,2250.00,33.934,8.57
482,1937,31.418030,-121.989970,0.000,33.70,0.00,7.15,323,144.0,0.829,0.00,78.890000,2.51,50.75,2270.19,34.049,7.18
483,1938,32.846330,-117.531300,0.037,0.75,0.05,13.95,30,307.5,0.117,0.05,259.410000,0.49,3.52,2225.36,33.285,13.95


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1454 entries, 0 to 1453
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 1454 non-null   int64  
 1   Lat_Dec            1454 non-null   float64
 2   Lon_Dec            1454 non-null   float64
 3   NO2uM              1454 non-null   float64
 4   NO3uM              1454 non-null   float64
 5   NH3uM              1454 non-null   float64
 6   R_TEMP             1454 non-null   float64
 7   R_Depth            1454 non-null   int64  
 8   R_Sal              1454 non-null   float64
 9   R_DYNHT            1454 non-null   float64
 10  R_Nuts             1454 non-null   float64
 11  R_Oxy_micromol.Kg  1454 non-null   float64
 12  Unnamed: 12        0 non-null      float64
 13  PO4uM              1454 non-null   float64
 14  SiO3uM             1454 non-null   float64
 15  TA1.x              1454 non-null   float64
 16  Salinity1          1454 

In [ ]:
sub

,id,DIC
0,1455,2150.46882
1,1456,2150.46882
2,1457,2150.46882
3,1458,2150.46882
4,1459,2150.46882
...,...,...
480,1935,2150.46882
481,1936,2150.46882
482,1937,2150.46882
483,1938,2150.46882


In [ ]:
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PowerTransformer
from sklearn.impute import SimpleImputer
import numpy as np

# Load data
# train = pd.read_csv("/kaggle/input/eds-232-ocean-chemistry-prediction-for-calcofi/train.csv")
# test = pd.read_csv("/kaggle/input/eds-232-ocean-chemistry-prediction-for-calcofi/test.csv")

# Drop unneeded columns and handle missing values
train = train.drop(columns=["Unnamed: 12", "id"])  # Dropping unnecessary columns

# Find common columns between train and test (excluding "DIC" from train)
common_columns = train.drop(columns=["DIC"]).columns.intersection(test.columns)

# Select the common columns for both train and test
X = train[common_columns]
y = train["DIC"]
test = test[common_columns]

# Split first to avoid leakage
X_train_raw, X_val_raw, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Target transform (log1p for stability)
TARGET_TRANSFORM = 'log1p'
y_train = np.log1p(y_train).astype(np.float32)
y_val = np.log1p(y_val).astype(np.float32)

# Impute missing values (fit on train only)
imputer = SimpleImputer(strategy='mean')
X_train_imp = imputer.fit_transform(X_train_raw)
X_val_imp = imputer.transform(X_val_raw)
test_imp = imputer.transform(test)

# Transform features with Yeo-Johnson (fit on train only)
pt = PowerTransformer(method='yeo-johnson', standardize=True)
X_train = pt.fit_transform(X_train_imp)
X_val = pt.transform(X_val_imp)
test_scaled = pt.transform(test_imp)

In [ ]:
from torch.utils.data import Dataset, DataLoader

class OceanChemistryDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y.values, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_dataset = OceanChemistryDataset(X_train, y_train)
val_dataset = OceanChemistryDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)


In [ ]:
class MLPModel(nn.Module):

    def __init__(self, input_size):

        super(MLPModel, self).__init__()

        self.fc1 = nn.Linear(input_size, 128)

        self.relu1 = nn.ReLU()

        self.fc3 = nn.Linear(128, 1)  # Output layer for regression


    def forward(self, x):

        x = self.fc1(x)

        x = self.relu1(x)

        x = self.fc3(x)

        return x


# Initialize the model

model = MLPModel(input_size=X_train.shape[1])

print(X_train.shape)


(1163, 15)


In [ ]:
import torch.optim as optim


# Loss function and optimizer

criterion = nn.MSELoss()

optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-4)

# LR scheduler
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=50, min_lr=1e-6)


# Training function

def train_model(model, train_loader, val_loader, epochs=5000, scheduler=None):

    for epoch in range(epochs):

        model.train()

        running_loss = 0.0


        for X_batch, y_batch in train_loader:

            optimizer.zero_grad()

            outputs = model(X_batch)

            loss = criterion(outputs.squeeze(), y_batch)

            loss.backward()

            optimizer.step()

            running_loss += loss.item()


        val_loss = 0.0

        model.eval()

        with torch.no_grad():

            for X_batch, y_batch in val_loader:

                outputs = model(X_batch)

                loss = criterion(outputs.squeeze(), y_batch)

                val_loss += loss.item()

        val_loss_avg = val_loss/len(val_loader) if len(val_loader)>0 else val_loss
        if scheduler is not None:
            scheduler.step(val_loss_avg)

        if epoch % 100 == 0:

            current_lr = optimizer.param_groups[0]['lr']
            print(f"Epoch {epoch+1}/{epochs}, LR: {current_lr:.2e}, Train Loss: {running_loss/len(train_loader)}, Validation Loss: {val_loss_avg}")


# Train the model

train_model(model, train_loader, val_loader, epochs=5000, scheduler=scheduler)


Epoch 1/5000, Train Loss: 4633887.947368421, Validation Loss: 4612593.2
Epoch 101/5000, Train Loss: 15084.370579769737, Validation Loss: 18933.8005859375
Epoch 201/5000, Train Loss: 2547.9099185341283, Validation Loss: 3594.0954345703126
Epoch 301/5000, Train Loss: 716.5324530350534, Validation Loss: 948.5962158203125
Epoch 401/5000, Train Loss: 400.37651142321135, Validation Loss: 575.134521484375
Epoch 501/5000, Train Loss: 284.9509711014597, Validation Loss: 413.65491943359376
Epoch 601/5000, Train Loss: 157.120555676912, Validation Loss: 215.35331115722656
Epoch 701/5000, Train Loss: 115.85318153782895, Validation Loss: 136.43557891845703
Epoch 801/5000, Train Loss: 78.71800462823165, Validation Loss: 102.47501983642579
Epoch 901/5000, Train Loss: 79.39923437018143, Validation Loss: 84.50692291259766
Epoch 1001/5000, Train Loss: 61.57915757831774, Validation Loss: 90.80420532226563
Epoch 1101/5000, Train Loss: 59.011202460841126, Validation Loss: 79.94369735717774
Epoch 1201/5000, 

In [ ]:
# Convert the test set into a torch tensor
test_tensor = torch.tensor(test_scaled, dtype=torch.float32)

# Set the model to evaluation mode
model.eval()

# Make predictions
with torch.no_grad():
    predictions = model(test_tensor).squeeze().numpy()

# Inverse target transform if applied
try:
    TARGET_TRANSFORM
except NameError:
    TARGET_TRANSFORM = None
if TARGET_TRANSFORM == 'log1p':
    predictions = np.expm1(predictions)

# Prepare submission
submission = pd.DataFrame({"id": range(1455, 1455 + len(predictions)), "DIC": predictions})
submission.to_csv("submission.csv", index=False)


In [ ]:
# score: 5.98852
# name: 杉浦孔明
# id: 123456

Python 3.10.14


In [ ]:
from sklearn.model_selection import KFold
from sklearn.preprocessing import PowerTransformer
from sklearn.impute import SimpleImputer

# K-Fold cross-validation (reduce seed dependence, avoid leakage)
n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=False)

test_pred_sum = np.zeros(len(test))

for fold, (trn_idx, val_idx) in enumerate(kf.split(X), 1):
    # Split raw features
    X_train_raw, X_val_raw = X.iloc[trn_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[trn_idx], y.iloc[val_idx]

    # Impute and transform per fold (fit only on training)
    imputer = SimpleImputer(strategy='mean')
    X_train_imp = imputer.fit_transform(X_train_raw)
    X_val_imp = imputer.transform(X_val_raw)
    test_imp = imputer.transform(test)

    pt = PowerTransformer(method='yeo-johnson', standardize=True)
    X_train_scaled = pt.fit_transform(X_train_imp)
    X_val_scaled = pt.transform(X_val_imp)
    test_scaled_fold = pt.transform(test_imp)

    # Target transform (log1p) within fold
    y_train = np.log1p(y_train).astype(np.float32)
    y_val = np.log1p(y_val).astype(np.float32)

    train_dataset = OceanChemistryDataset(X_train_scaled, y_train)
    val_dataset = OceanChemistryDataset(X_val_scaled, y_val)
    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

    model = MLPModel(input_size=X.shape[1])
    optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-4)

    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=50, min_lr=1e-6)

    print(f'Fold {fold}/{n_splits}')
    train_model(model, train_loader, val_loader, epochs=5000, scheduler=scheduler)

    model.eval()
    with torch.no_grad():
        fold_pred_t = model(torch.tensor(test_scaled_fold, dtype=torch.float32)).squeeze().numpy()
    # Inverse target transform
    fold_pred = np.expm1(fold_pred_t)
    test_pred_sum += fold_pred

# Averaged predictions across folds
predictions = test_pred_sum / n_splits

# Prepare submission (overwrites with K-Fold average)
submission = pd.DataFrame({'id': range(1455, 1455 + len(predictions)), 'DIC': predictions})
submission.to_csv('submission.csv', index=False)
